## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database_copy.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,La Rioja,ES,42.2500,-2.5000,60.80,36,0,9.22,clear sky
1,1,Sinnamary,GF,5.3833,-52.9500,79.50,85,100,10.31,light rain
2,2,Chipata,ZM,-13.6333,32.6500,64.44,89,53,5.50,broken clouds
3,3,Namibe,AO,-15.1961,12.1522,78.53,87,98,9.71,overcast clouds
4,4,Hasaki,JP,35.7333,140.8333,64.40,94,75,6.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sinnamary,GF,5.3833,-52.9500,79.50,85,100,10.31,light rain
3,3,Namibe,AO,-15.1961,12.1522,78.53,87,98,9.71,overcast clouds
12,12,Port Moresby,PG,-9.4431,147.1797,77.00,94,40,3.44,scattered clouds
15,15,Rikitea,PF,-23.1203,-134.9692,76.62,70,29,15.84,light rain
17,17,Bengkulu,ID,-3.8004,102.2655,76.44,87,100,8.97,overcast clouds
18,18,Georgetown,MY,5.4112,100.3354,84.99,83,20,4.45,heavy intensity rain
20,20,Kieta,PG,-6.2167,155.6333,77.81,84,96,4.52,light rain
23,23,Bambous Virieux,MU,-20.3428,57.7575,81.00,74,14,5.99,few clouds
24,24,Iralaya,HN,15.0000,-83.2333,77.00,94,90,13.80,light intensity drizzle
26,26,Atuona,PF,-9.8000,-139.0333,80.40,81,59,11.63,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                256
City                   256
Country                254
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                254
City                   254
Country                254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sinnamary,GF,79.50,light rain,5.3833,-52.9500,
3,Namibe,AO,78.53,overcast clouds,-15.1961,12.1522,
12,Port Moresby,PG,77.00,scattered clouds,-9.4431,147.1797,
15,Rikitea,PF,76.62,light rain,-23.1203,-134.9692,
17,Bengkulu,ID,76.44,overcast clouds,-3.8004,102.2655,
18,Georgetown,MY,84.99,heavy intensity rain,5.4112,100.3354,
20,Kieta,PG,77.81,light rain,-6.2167,155.6333,
23,Bambous Virieux,MU,81.00,few clouds,-20.3428,57.7575,
24,Iralaya,HN,77.00,light intensity drizzle,15.0000,-83.2333,
26,Atuona,PF,80.40,broken clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

City                   254
Country                254
Max Temp               254
Current Description    254
Lat                    254
Lng                    254
Hotel Name             254
dtype: int64

In [12]:
# Create the output file (CSV).
output_data_file = "Vacation_Search.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))